prepare dataset

In [1]:
from train import prepare_dataset

INPUT_DIR = ''
ALIGNED_DIR = ''
MASKED_DIR = ''
COMBINE_DIR = ''

print('INFO: PREPARE DATASET')

print('INFO: Align face')
print('Please wait...')
prepare_dataset.aligned_dataset(INPUT_DIR, ALIGNED_DIR)
print('Aligned face save at ' + ALIGNED_DIR)
print('INFO: Remove face mask region')
print('Please wait...')
prepare_dataset.masked_dataset(ALIGNED_DIR, MASKED_DIR)
print('Masked face save at ' + MASKED_DIR)
print('INFO: Combine dataset')
print('Please wait...')
prepare_dataset.combine_dataset(ALIGNED_DIR, MASKED_DIR, COMBINE_DIR)
print('Combine dataset save at ' + COMBINE_DIR)

INFO: PREPARE DATASET
INFO: Align face
Please wait...
Aligned face save at dataset/celeb_vn/aligned
INFO: Remove face mask region
Please wait...
Masked face save at dataset/celeb_vn/masked
INFO: Combine dataset
Please wait...
Combine dataset save at dataset/celeb_vn/combine


In [2]:
import os
import cv2
in_dir = ''
DATA_DIR = ''
for fld_name in os.listdir(in_dir):
    fld_path = os.path.join(in_dir, fld_name)
    new_fld_path = os.path.join(DATA_DIR, fld_name)
    os.mkdir(new_fld_path)
    for file_name in os.listdir(fld_path):
        file_path = os.path.join(fld_path, file_name)
        new_file_path = os.path.join(new_fld_path, file_name)
        img = cv2.imread(file_path)
        center = (img.shape[1]//2, img.shape[0]//2)
        pad_x = int(12.5*img.shape[1]/100)
        pad_y = int(12.5*img.shape[0]/100)
        ret_img = img.copy()[pad_y:img.shape[1]-pad_y, pad_x:img.shape[0]-pad_x]
        cv2.imwrite(new_file_path, ret_img)

train model

In [2]:
import tensorflow as tf
import keras
from keras.metrics import CategoricalAccuracy
from keras.preprocessing.image import ImageDataGenerator
import os
import sys
sys.path.append(r'D:\sw\face_rec\face_reg_new_ui')
from models.feature_extraction_model.inceptionresnetv2 import get_train_model
from keras.models import Model
from train.arcface_metrics import ArcFace
from keras.layers import Input
import numpy as np
import math


EPOCHS = 200
BATCH_SIZE = 256
DATA_DIR = ''
RESULTS_DIR = 'train/results'

print('INFO: TRAIN MODEL')

print('INFO: Dataset pre_processing')
print('Please wait...')

num_class = len(os.listdir(DATA_DIR))

train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    label_mode = 'categorical',
    image_size=(160, 160),
    batch_size=BATCH_SIZE,
    shuffle=True,
    subset = "training",
    validation_split = 0.2,
    seed = 23,
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    label_mode = 'categorical',
    image_size=(160, 160),
    batch_size=BATCH_SIZE,
    shuffle=True,
    subset = "validation",
    validation_split = 0.2,
    seed = 23,
)

train_ds = train_ds.prefetch(buffer_size=BATCH_SIZE)
val_ds = val_ds.prefetch(buffer_size=BATCH_SIZE)

# data_augmentation = keras.layers.RandomZoom(0.2)
# augmented_train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))
normalization_layer = keras.layers.Rescaling(1./255)
normalized_train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
normalized_val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

print('INFO: Create model')
model = get_train_model(num_class)

print('INFO: Compile model')
model.compile(loss='categorical_crossentropy',optimizer='Adam', metrics=[CategoricalAccuracy()])

cp1 = keras.callbacks.ModelCheckpoint(os.path.join(RESULTS_DIR, "check_point_.h5"), monitor='loss', verbose=1, save_best_only=True, save_weights_only=False, mode='min')

callbacks = [
    cp1,
]

model.summary()

history = model.fit(
    normalized_train_ds,
    epochs = EPOCHS,
    verbose = 1,
    callbacks = callbacks,
    validation_data = normalized_val_ds,
    shuffle=True,
)

model.save(os.path.join(RESULTS_DIR, "final_.h5"))

INFO: TRAIN MODEL
INFO: Dataset pre_processing
Please wait...
Found 7352 files belonging to 515 classes.
Using 5882 files for training.
Found 7352 files belonging to 515 classes.
Using 1470 files for validation.
INFO: Create model
INFO: Compile model
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 299, 299, 3)]     0         
                                                                 
 inception_resnet_v2 (Functi  (None, 8, 8, 1536)       54336736  
 onal)                                                           
                                                                 
 global_average_pooling2d (G  (None, 1536)             0         
 lobalAveragePooling2D)                                          
                                                                 
 flatten (Flatten)           (None, 1536)              0         
        

ResourceExhaustedError: Graph execution error:

Detected at node 'model/inception_resnet_v2/batch_normalization_122/FusedBatchNormV3' defined at (most recent call last):
    File "C:\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "d:\sw\face_rec\face_reg_new_ui\env\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "d:\sw\face_rec\face_reg_new_ui\env\lib\site-packages\traitlets\config\application.py", line 976, in launch_instance
      app.start()
    File "d:\sw\face_rec\face_reg_new_ui\env\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "d:\sw\face_rec\face_reg_new_ui\env\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "C:\Python310\lib\asyncio\base_events.py", 

convert trained model to embedding model

softmax head

In [1]:
from models.feature_extraction_model.inceptionresnetv2 import get_train_model
from keras.models import Model, load_model

base_model = load_model('train/results/check_point.h5')
model = Model(base_model.inputs, base_model.layers[-2].output)
model.save_weights('models/feature_extraction_model/inceptionresnetv2_512_weights.h5')
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 160, 160, 3)]     0         
                                                                 
 inception_resnet_v2 (Functi  (None, 3, 3, 1536)       54336736  
 onal)                                                           
                                                                 
 global_average_pooling2d (G  (None, 1536)             0         
 lobalAveragePooling2D)                                          
                                                                 
 flatten (Flatten)           (None, 1536)              0         
                                                                 
 dense (Dense)               (None, 1024)              1572864   
                                                                 
 dropout (Dropout)           (None, 1024)              0     

arcface head

In [1]:
from keras.models import Model, load_model
from train.arcface_metrics import ArcFace

model = load_model(r'train\results\check_point.h5', custom_objects={'ArcFace': ArcFace})
model = model.get_layer('model_2')
model.summary()
model.save(r'models/feature_extraction_model/inceptionresnetv2_512_arcface.h5')

d:\sw\face_rec\face_reg_new_ui\env\lib\site-packages\keras\layers\core\lambda_layer.py:303: UserWarning: keras.applications.inception_resnet_v2 is not loaded, but a Lambda layer uses it. It may cause errors.
  function = cls._parse_function_from_config(config, custom_objects,


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 160, 160, 3)]     0         
                                                                 
 inception_resnet_v2 (Functi  (None, 3, 3, 1536)       54336736  
 onal)                                                           
                                                                 
 AvgPool (GlobalAveragePooli  (None, 1536)             0         
 ng2D)                                                           
                                                                 
 Dropout (Dropout)           (None, 1536)              0         
                                                                 
 Bottleneck (Dense)          (None, 512)               786432    
                                                                 
 Bottleneck_BatchNorm (Batch  (None, 512)              1536